step 1. select potential

In [ ]:
from calorine.calculators import CPUNEP

nep_path = '../Repository/UNEP_v1/nep.txt'
calc = NEP('../Repository/UNEP_v1/nep.txt')
symbols = ['Al' , 'Au' , 'Cr' , 'Cu' , 'Mg' , 'Mo' , 'Ni' , 'Pb' , 'Pd' , 'Pt' , 'Ta' , 'Ti' , 'V' , 'W' , 'Zr']

step 2. deform and perturbation

In [ ]:
from wizard.frames import MultiMol
from wizard.atoms import SymbolInfo
from wizard.io import relax
from itertools import combinations

binary = [''.join(comb) for comb in combinations(symbols, 2)]
formulars = symbols + binary

symbol_infos = []
for formular in formulars:
    symbol_infos.append(SymbolInfo(formular, 'bcc', 3))
    symbol_infos.append(SymbolInfo(formular, 'fcc', 3))
    symbol_infos.append(SymbolInfo(formular, 'hcp', 3,5))

frames = []
for symbol_info in symbol_infos:
    atoms = symbol_info.create_bulk_atoms((2, 2, 2))
    atoms.calc = calc
    relax(atoms)
    frames.append(atoms)

deform_scale = np.arange(0.95, 1.06, 0.05)
strain_ratio = 0.04
max_displacement = 0.4

deform = MultiMol(frames).deform(deform_scale)
deform_scale = MultiMol(deform).random_strain(strain_ratio)
deform_pertu = MultiMol(deform).random_displacement(max_displacement)

frames = deform + deform_scale + deform_pertu
MultiMol(frames).dump('deform.xyz')

step 3. material properties

In [ ]:
from wizard.calculator import MaterialCalculator
from wizard.atoms import SymbolInfo
from itertools import combinations

binary = [''.join(comb) for comb in combinations(symbols, 2)]
formulars = symbols + binary

symbol_infos = []
for formular in formulars:
   symbol_infos.append(SymbolInfo(formular, 'bcc', 3))
   symbol_infos.append(SymbolInfo(formular, 'fcc', 3))
   symbol_infos.append(SymbolInfo(formular, 'hcp', 3,5))

nths = [1,2,3,4,5]
sia_vectors = [(1/2,1/2,1/2),(1,0,0),(1,1,0)]
millers = [(1,1,0),(0,0,1),(1,1,1),(1,1,2)]

for symbol_info in symbol_infos:
   atoms = symbol_info.create_bulk_atoms()
   material_calculator = MaterialCalculator(atoms, calc, symbol_info)
   material_calculator.formation_energy_vacancy()
   material_calculator.migration_energy_vacancy()

for symbol_info in symbol_infos:
   atoms = symbol_info.create_bulk_atoms((3,4,5))
   for nth in nths:
      material_calculator.formation_energy_divacancies(nth)

for symbol_info in symbol_infos:
   atoms = symbol_info.create_bulk_atoms((3,4,5))
   material_calculator = MaterialCalculator(atoms, calc, symbol_info)
   for vector in sia_vectors:
      material_calculator.formation_energy_sia(vector)

for symbol_info in symbol_infos:
   atoms = symbol_info.create_bulk_atoms((1,1,1))
   material_calculator = MaterialCalculator(atoms, calc, symbol_info)
   for miller in millers:
      material_calculator.formation_energy_surface(miller)

step 4. vacancies

In [ ]:
from wizard.atoms import SymbolInfo, Morph
from wizard.frames import MultiMol
from itertools import combinations

binary = [''.join(comb) for comb in combinations(symbols, 2)]
formulars = symbols + binary

symbol_infos = []
for formular in formulars:
   symbol_infos.append(SymbolInfo(formular, 'bcc', 3))
   symbol_infos.append(SymbolInfo(formular, 'fcc', 3))
   symbol_infos.append(SymbolInfo(formular, 'hcp', 3,5))

frames = []
for symbol_info in symbol_infos:
    for n in range(3, 10, 3):
        if symbol_info.structure == 'bcc':
            atoms = symbol_info.create_bulk_atoms()
        elif symbol_info.structure == 'fcc':
            atoms = symbol_info.create_bulk_atoms((2, 2, 2))
        elif symbol_info.structure == 'hcp':
            atoms = symbol_info.create_bulk_atoms((3, 3, 2)) 
        Morph(atoms).create_vacancies(n)
        frames.append(atoms)
MultiMol(frames).dump('vacancies.xyz')

step 5. npt sample

In [ ]:
from wizard.atoms import SymbolInfo, Morph
from wizard.io import read_xyz
from wizard.frames import MultiMol
from itertools import combinations

binary = [''.join(comb) for comb in combinations(symbols, 2)]
formulars = symbols + binary

symbol_infos = []
for formular in formulars:
   symbol_infos.append(SymbolInfo(formular, 'bcc', 3))
   symbol_infos.append(SymbolInfo(formular, 'fcc', 3))
   symbol_infos.append(SymbolInfo(formular, 'hcp', 3,5))

Temp_Struts = []
temperatures = [50, 300, 800, 1300, 1700, 2300, 3000, 4000, 5000]
for symbol_info in symbol_infos:
    for temperature in temperatures:
        atoms = symbol_info.create_bulk_atoms((3,3,3))
        dirname = f'{symbol_info.formula}/{symbol_info.structure}/{temperature}K/relax'
        run_in=['potential nep.txt',
                f'velocity {temperature}',   
                'time_step 1',
                f'ensemble npt_mttk temp {temperature} {temperature} iso 0 0',
                'dump_thermo 10000',
                'dump_exyz 200000',
                'dump_restart 1000000',
                'run 1000000']
        Morph(atoms).gpumd(dirname=dirname, nep_path= nep_path, run_in=run_in)
        Temp_Struts += read_xyz(dirname + '/dump.xyz')

deform_scale = np.arange(0.95, 1.06, 0.05)
strain_ratio = 0.05
Temp_Struts_Deform = MultiMol(Temp_Struts).deform(deform_scale)
Temp_Struts_Strain = MultiMol(Temp_Struts).random_strain(strain_ratio)

Temp_Struts = Temp_Struts_Deform + Temp_Struts_Strain
MultiMol(Temp_Struts).dump('Temp_Struts.xyz')

step 6. utc sample

In [ ]:
from wizard.atoms import SymbolInfo, Morph
from wizard.io import read_restart, read_xyz
from wizard.frames import MultiMol
from itertools import combinations

binary = [''.join(comb) for comb in combinations(symbols, 2)]
formulars = symbols + binary

symbol_infos = []
for formular in formulars:
   symbol_infos.append(SymbolInfo(formular, 'bcc', 3))
   symbol_infos.append(SymbolInfo(formular, 'fcc', 3))
   symbol_infos.append(SymbolInfo(formular, 'hcp', 3,5))

temperature = 300 #K
strain_rate = 2e8
dt = 1e-15 #fs
utc_tensile = []
for symbol_info in symbol_infos:
    atoms = read_restart(f'{symbol_info.symbol}/{symbol_info.structure}/{temperature}K/relax/restart.xyz')
    dirname = f'{symbol_info.symbol}/{symbol_info.structure}/{temperature}K/utc_tensile'
    length = atoms.cell[2, 2]
    strain = strain_rate * dt * length
    run_in = ['potential ../../../../nep.txt', 
              'velocity 300', 
              'time_step 1',
              f'ensemble npt_scr {temperature} {temperature} 100 0 0 0 100 100 100 1000',
              f'deform {strain} 0 0 1', 
              'dump_thermo 1000', 
              'dump_exyz 200000', 
              'dump_restart 10000',
              'run 2000000']
    Morph(atoms).gpumd(dirname=dirname, nep_path= nep_path, run_in=run_in)
    utc_tensile += read_xyz(dirname + '/dump.xyz') 

MultiMol(utc_tensile).dump('utc_tensile.xyz')

step 7. ttc sample

In [ ]:
from wizard.atoms import SymbolInfo, Morph
from wizard.io import read_restart, read_xyz
from wizard.frames import MultiMol
from itertools import combinations

binary = [''.join(comb) for comb in combinations(symbols, 2)]
formulars = symbols + binary

symbol_infos = []
for formular in formulars:
   symbol_infos.append(SymbolInfo(formular, 'bcc', 3))
   symbol_infos.append(SymbolInfo(formular, 'fcc', 3))
   symbol_infos.append(SymbolInfo(formular, 'hcp', 3,5))

temperature = 300 #K
strain_rate = 2e8
dt = 1e-15 #fs
ttc_tensile = []
for symbol_info in symbol_infos:
    atoms = read_restart(f'{symbol_info.symbol}/{symbol_info.structure}/{temperature}K/relax/restart.xyz')
    dirname = f'{symbol_info.symbol}/{symbol_info.structure}/{temperature}K/ttc_tensile'
    length = atoms.cell[2, 2]
    strain = strain_rate * dt * length
    run_in = ['potential ../../../../nep.txt', 
              'velocity 300', 
              'time_step 1',
              f'ensemble npt_scr {temperature} {temperature} 100 0 0 0 100 100 100 1000',
              f'deform {strain} 1 1 1', 
              'dump_thermo 1000', 
              'dump_exyz 200000', 
              'dump_restart 10000',
              'run 2000000']
    Morph(atoms).gpumd(dirname=dirname, nep_path= nep_path, run_in=run_in)
    ttc_tensile += read_xyz(dirname + '/dump.xyz') 

MultiMol(ttc_tensile).dump('ttc_tensile.xyz')